In [1]:
import array, csv, json, math, multiprocessing, os, random, re, shutil
import shapely, shapely.wkb, struct, subprocess, sys, threading, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [26]:
src = 'od-JT01-2011.bin'
outputdir = 'od-JT01-2011-tiles'

In [27]:
# merc_x and merc_y are in 0-256 web mercator space
def MercToTileCoords(merc_x, merc_y, z):
    n = 2 ** z
    xtile = int(merc_x * n / 256)
    ytile = int(merc_y * n / 256)
    return xtile, ytile

In [28]:
def make_tile_zoom(z, subsample):
    tiles = [[[] for x in range(2 ** z)] for y in range(2 ** z)]

    record_len = 6 * 4

    num_records = os.stat(src).st_size / record_len
    
    print '%s has %d records (points)' % (src, num_records)
    if subsample < 1:
        num_records = int(round(num_records * subsample))
        print 'Subsampling %g%% to %d records (points)' % (subsample * 100, num_records)
    
    src_file = open(src, 'rb')

    for i in xrange(num_records):
        record = src_file.read(record_len)
        (wx, wy, hx, hy, dist, color) = struct.unpack('<ffffff', record)
        (xtile, ytile) = MercToTileCoords(wx, wy, z)
        tiles[xtile][ytile].append(record)
    
    for ytile in range(2 ** z):
        for xtile in range(2 ** z):
            tile_path = "%s/%d/%d/%d.bin" % (outputdir, z, xtile, ytile)
            if not os.path.exists(os.path.dirname(tile_path)):
                os.makedirs(os.path.dirname(tile_path))
            open(tile_path, 'wb').write(''.join(tiles[xtile][ytile]))

    print 'Created %d tiles with prefix %s/%d' % (4 ** z, outputdir, z)

In [29]:
make_tile_zoom( 0, 0.000244140625)
make_tile_zoom( 1, 0.000244140625)
make_tile_zoom( 2, 0.000244140625)
make_tile_zoom( 3, 0.000244140625)
make_tile_zoom( 4, 0.000244140625)
make_tile_zoom( 5, 0.0009765625)
make_tile_zoom( 6, 0.00390625)
make_tile_zoom( 7, 0.015625)
make_tile_zoom( 8, 0.0625)
make_tile_zoom( 9, 0.25)
make_tile_zoom(10, 1.0)


od-JT01-2011.bin has 223234100 records (points)
Subsampling 0.0244141% to 54501 records (points)
Created 1 tiles with prefix od-JT01-2011-tiles/0
od-JT01-2011.bin has 223234100 records (points)
Subsampling 0.0244141% to 54501 records (points)
Created 4 tiles with prefix od-JT01-2011-tiles/1
od-JT01-2011.bin has 223234100 records (points)
Subsampling 0.0244141% to 54501 records (points)
Created 16 tiles with prefix od-JT01-2011-tiles/2
od-JT01-2011.bin has 223234100 records (points)
Subsampling 0.0244141% to 54501 records (points)
Created 64 tiles with prefix od-JT01-2011-tiles/3
od-JT01-2011.bin has 223234100 records (points)
Subsampling 0.0244141% to 54501 records (points)
Created 256 tiles with prefix od-JT01-2011-tiles/4
od-JT01-2011.bin has 223234100 records (points)
Subsampling 0.0976562% to 218002 records (points)
Created 1024 tiles with prefix od-JT01-2011-tiles/5
od-JT01-2011.bin has 223234100 records (points)
Subsampling 0.390625% to 872008 records (points)
Created 4096 tiles 

In [21]:
!du -shk tiles/5/*

4	tiles/5/0
4	tiles/5/1
12	tiles/5/10
892	tiles/5/11
4620	tiles/5/12
16	tiles/5/13
12	tiles/5/14
4	tiles/5/15
4	tiles/5/16
4	tiles/5/17
4	tiles/5/18
4	tiles/5/19
4	tiles/5/2
4	tiles/5/20
4	tiles/5/21
4	tiles/5/22
4	tiles/5/23
4	tiles/5/24
4	tiles/5/25
4	tiles/5/26
4	tiles/5/27
4	tiles/5/28
4	tiles/5/29
4	tiles/5/3
4	tiles/5/30
4	tiles/5/31
4	tiles/5/4
4	tiles/5/5
4	tiles/5/6
8	tiles/5/7
12	tiles/5/8
12	tiles/5/9
